In [1]:
from pyspark import SparkContext
import os
notebook_path = os.path.abspath("Notebook.ipynb")
albums = os.path.join(os.getcwd(), "project1/data", "albums.csv")
artists = os.path.join(os.getcwd(), "project1/data", "artists.csv")

In [2]:
sc = SparkContext("local", "Big D")

In [3]:
albums_raw = sc.textFile(albums)  # one list with all the data
albums_rdd = albums_raw.map(lambda x: x.split(','))  # proper csv format (list of lists)

artists_raw = sc.textFile(artists)
artists_rdd = artists_raw.map(lambda x: x.split(','))

In [4]:
fields = 'id artist_id album_title genre year_of_pub num_of_tracks num_of_sales rolling_stone_critic mtv_critic music_maniac_critic'.split()
print fields
def task_1():
    print albums_rdd.map(lambda x: x[3]).distinct().count()
task_1()


['id', 'artist_id', 'album_title', 'genre', 'year_of_pub', 'num_of_tracks', 'num_of_sales', 'rolling_stone_critic', 'mtv_critic', 'music_maniac_critic']


38


In [5]:
artists_fields = 'id real_name art_name role year_of_birth country city email zip_code'.split()
artists_fields


['id',
 'real_name',
 'art_name',
 'role',
 'year_of_birth',
 'country',
 'city',
 'email',
 'zip_code']

In [6]:
def task_2():
    print artists_rdd.sortBy(lambda x: x[4]).first()[4]
task_2()

1955


In [12]:
def count_column_val(rdd, col):
    return rdd.map(lambda x: x[col]).countByValue()

def sort_key_val(data, key_is_num=False):
    sort = lambda(k,v): (-v, k)
    if key_is_num:
        # make sure we sort by int value and not str
        # e.g "764">"6082"
        sort = lambda(k,v): (-v, int(k))
    return [x for x in sorted(data.items(), key=sort)]

def rdd(data):
    return sc.parallelize(data)

def delimit(*args):
    parsed = []
    for arg in args:
        tmp = arg
        if type(arg) == int or type(arg) == float:
            tmp = str(arg)
        elif type(arg) != str:
            tmp = arg.encode('utf-8')
        parsed.append(tmp)       
    return '\t'.join(parsed)

def save_as_tsv(data, filename):
    as_rdd = rdd(data)
    delimited = as_rdd.map(lambda (k,v): delimit(k, v))
    delimited.saveAsTextFile(os.path.join(os.getcwd(), filename))
    

In [13]:
def task_3():
    COL_COUNTRY = 5
    sorted_by_country = sort_key_val(count_column_val(artists_rdd, COL_COUNTRY))
    save_as_tsv(sorted_by_country, 'result_3.tsv')
task_3()

In [9]:
def task_4():
    COL_ARTIST_ID = 1
    sorted_by_albums = sort_key_val(count_column_val(albums_rdd, COL_ARTIST_ID), key_is_num=True)
    save_as_tsv(sorted_by_albums, 'result_4.tsv')
task_4()

In [10]:
def task_5():
    COL_GENRE, COL_NUM_SALES = 3, 6
    genre_sales = lambda x: (x[COL_GENRE], x[COL_NUM_SALES])
    add = lambda a, b: int(a) + int(b)
    
    sales_per_genre = albums_rdd.map(genre_sales).reduceByKey(add).collectAsMap()
    sorted_sales = sort_key_val(sales_per_genre)
    save_as_tsv(sorted_sales, 'result_5.tsv')
task_5()

In [11]:
def task_6():
    ID = 0
    ROLLING_STONE, MTV, MUSIC_MANIAC = 7, 8, 9
    get_critics = lambda x: (x[ID], (
        float(x[ROLLING_STONE]) + float(x[MTV]) + float(x[MUSIC_MANIAC]))/3)
    avg_critic = albums_rdd.map(get_critics).takeOrdered(10, lambda (_id, avg): -avg)
    save_as_tsv(avg_critic, 'result_6.tsv')
task_6()

In [12]:
def task_7():
    